# Generating Embeddings

We can generate embeddings in many different ways, word2vec, BERT, ada-002

## Generating embeddings with Ada-002 from OpenAI

In [1]:
%pip install openai


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import os

openai.api_key = os.getenv("OPEN_AI_API_KEY")
MODEL = "text-embedding-ada-002"

In [3]:
# Generate embeddings for sentences
sentences = ["Hello, how are you?", "I am doing great.", "What's your name?"]
embeddings_response = openai.Embedding.create(input=sentences, engine=MODEL)
embeddings = [record['embedding'] for record in embeddings_response['data']]

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embedding}")
    print()


Sentence: Hello, how are you?
Embedding: [-0.008576292544603348, -0.0004712930240202695, 0.003618414280936122, -0.03318895027041435, -0.012024173513054848, 0.018963342532515526, -0.00927703082561493, -0.009196415543556213, -0.017289012670516968, -0.010678508318960667, 0.03127897530794144, 0.010895551182329655, -0.016792915761470795, -0.008557689376175404, 0.007832146249711514, -0.015850327908992767, 0.02669006772339344, -0.0076523106545209885, 0.029319386929273605, -0.01305977813899517, -0.02124539390206337, 0.00345408171415329, 0.018244002014398575, -0.0018913733074441552, 0.0019533855374902487, -0.010368446819484234, 0.016458049416542053, -0.016123183071613312, 0.016408439725637436, -0.02631799504160881, 0.005367159377783537, -0.000988320098258555, -0.007379456888884306, -0.004436975810676813, 0.011906350031495094, -0.021493442356586456, 0.0010332789970561862, -0.009413458406925201, 0.023006541654467583, -0.015267414040863514, 0.022795699536800385, -0.0015634836163371801, 0.013158997

### OpenAI ada-002 is a language model that can be used to generate embeddings. Its embeddings have 1536 dimensions.